![Image](https://www.geeky-gadgets.com/wp-content/uploads/2024/04/Phi-3-AI-small-language-models.webp)

# Overview

- RAG (Retrieval Augmented Generation) with [Phi-3](https://huggingface.co/microsoft/Phi-3-mini-128k-instruct)
- Data: [100 LLM Papers to explore](https://www.kaggle.com/datasets/ruchi798/100-llm-papers-to-explore)
- Works in a single P100 or in dual-T4
- You don't need any token or key to use Phi-3! 😀

# Imports

In [1]:
! nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-bfc0dba3-8b0f-2845-c77a-c36f9ed217b3)
GPU 1: Tesla T4 (UUID: GPU-f1f5e3c5-a901-10d2-97dc-34cdfb39415e)


In [2]:
%%time

from IPython.display import clear_output

! pip install -q -U langchain tiktoken pypdf faiss-gpu sentence_transformers==2.2.2 InstructorEmbedding transformers accelerate bitsandbytes

clear_output()

CPU times: user 945 ms, sys: 170 ms, total: 1.11 s
Wall time: 52.8 s


In [3]:
%%time

import warnings
warnings.filterwarnings("ignore")

import os
import glob
import textwrap
import time
import gc

import langchain

# loaders
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

# splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

# prompts
from langchain import PromptTemplate

# vector stores
from langchain_community.vectorstores import FAISS

# models
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceInstructEmbeddings

# retrievers
from langchain.chains import RetrievalQA

import torch

import transformers
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)

clear_output()

CPU times: user 12.6 s, sys: 2.02 s, total: 14.6 s
Wall time: 24 s


In [4]:
len(glob.glob('/kaggle/input/100-llm-papers-to-explore/*'))

100

# CFG

- CFG class enables easy and organized experimentation

In [5]:
class CFG:
    DEBUG = False
    
    ### LLM
    model_name = 'microsoft/Phi-3-mini-128k-instruct'
    temperature = 0.4
    top_p = 0.90
    repetition_penalty = 1.15
    max_len = 8192
    max_new_tokens = 512

    ### splitting
    split_chunk_size = 800
    split_overlap = 400
    
    ### embeddings
    embeddings_model_repo = 'BAAI/bge-base-en-v1.5'

    ### similar passages
    k = 6
    
    ### paths
    PDFs_path = '/kaggle/input/100-llm-papers-to-explore/'
    Embeddings_path =  '/kaggle/input/faiss-ml-papers-st'
    Output_folder = './ml-papers-vectordb'

# Load data

In [6]:
%%time

loader = DirectoryLoader(
    CFG.PDFs_path,
    glob = "./*3215v3.pdf" if CFG.DEBUG else "./*.pdf",
    loader_cls = PyPDFLoader,
    show_progress = True,
    use_multithreading = True
)

documents = loader.load()

100%|██████████| 100/100 [06:30<00:00,  3.91s/it]

CPU times: user 6min 30s, sys: 8.67 s, total: 6min 39s
Wall time: 6min 30s


In [7]:
print(f'We have {len(documents)} pages in total')

We have 2871 pages in total


# Splitter

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = CFG.split_chunk_size,
    chunk_overlap = CFG.split_overlap
)

texts = text_splitter.split_documents(documents)

print(f'We have created {len(texts)} chunks from {len(documents)} pages')

We have created 23268 chunks from 2871 pages


# Create Embeddings

In [9]:
%%time

### we create the embeddings if they do not already exist in the input folder
if not os.path.exists(CFG.Embeddings_path + '/index.faiss'):
    
    print('Creating embeddings...\n\n')

    ### download embeddings model
    embeddings = HuggingFaceInstructEmbeddings(
        model_name = CFG.embeddings_model_repo,
        model_kwargs = {"device": "cuda"}
    )

    ### create embeddings and DB
    vectordb = FAISS.from_documents(
        documents = texts, 
        embedding = embeddings
    )

    ### persist vector database
    vectordb.save_local(f"{CFG.Output_folder}/faiss_index_ml_papers") # save in output folder
#     vectordb.save_local(f"{CFG.Embeddings_path}/faiss_index_ml_papers") # save in input folder

clear_output()

CPU times: user 8min 35s, sys: 3.87 s, total: 8min 39s
Wall time: 8min 25s


# Load Embeddings

- You can just load the embeddings if you have already created and saved them previously

In [10]:
%%time

### download embeddings model
embeddings = HuggingFaceInstructEmbeddings(
    model_name = CFG.embeddings_model_repo,
    model_kwargs = {"device": "cuda"}
)

### load vector DB embeddings
vectordb = FAISS.load_local(
#     CFG.Embeddings_path, # from input folder
    CFG.Output_folder + '/faiss_index_ml_papers', # from output folder
    embeddings,
    allow_dangerous_deserialization = True,
)

clear_output()

CPU times: user 924 ms, sys: 323 ms, total: 1.25 s
Wall time: 1.04 s


In [11]:
%%time

### test if vector DB was loaded correctly
vectordb.similarity_search('scaling laws')

CPU times: user 168 ms, sys: 7.05 ms, total: 175 ms
Wall time: 215 ms


[Document(page_content='itory , 2020. doi: 10.48550/arXiv.2010.14701. URL\nhttps://arxiv.org/abs/2010.14701v2 . Ver-\nsion 2.\nHernandez, D., Kaplan, J., Henighan, T., and McCandlish,\nS. Scaling laws for transfer. Computing Research Repos-\nitory , 2021. doi: 10.48550/arXiv.2102.01293. URL\nhttps://arxiv.org/abs/2102.01293v1 . Ver-\nsion 1.\nHernandez, D., Brown, T., Conerly, T., DasSarma, N.,\nDrain, D., El-Showk, S., Elhage, N., Hatfield-Dodds,\nZ., Henighan, T., Hume, T., Johnston, S., Mann, B., Olah,\nC., Olsson, C., Amodei, D., Joseph, N., Kaplan, J., and\nMcCandlish, S. Scaling laws and interpretability of learn-\ning from repeated data. Computing Research Reposi-\ntory, 05 2022. doi: 10.48550/arXiv.2205.10487. URL\nhttps://arxiv.org/abs/2205.10487v1 . Ver-\nsion 1.', metadata={'source': '/kaggle/input/100-llm-papers-to-explore/2304.01373.pdf', 'page': 11}),
 Document(page_content='Contents\n1 Introduction 3\n2 Scaling law experiments 7\n2.1 Scaling laws . . . . . . . . . . . . 

# Model

In [12]:
def build_model(model_repo = CFG.model_name):

    print('\nDownloading model: ', model_repo, '\n\n')

    ### tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_repo)

    ### quantization
    bnb_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_quant_type = "nf4",
        bnb_4bit_compute_dtype = torch.float16,
        bnb_4bit_use_double_quant = True,
    )        

    ### model
    model = AutoModelForCausalLM.from_pretrained(
        model_repo,
        quantization_config = bnb_config,
        device_map = 'auto',
        low_cpu_mem_usage = True,
        trust_remote_code = True,
    )

    return tokenizer, model

In [13]:
%%time

tokenizer, model = build_model(model_repo = CFG.model_name)

clear_output()

CPU times: user 19.9 s, sys: 16.4 s, total: 36.3 s
Wall time: 39.9 s


In [14]:
gc.collect()

51

In [15]:
model.eval()

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3SuScaledRotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features

In [16]:
### check how Accelerate split the model across the available devices (GPUs)
model.hf_device_map

{'model.embed_tokens': 0,
 'model.embed_dropout': 0,
 'model.layers.0': 0,
 'model.layers.1': 0,
 'model.layers.2': 0,
 'model.layers.3': 0,
 'model.layers.4': 0,
 'model.layers.5': 0,
 'model.layers.6': 0,
 'model.layers.7': 0,
 'model.layers.8': 0,
 'model.layers.9': 0,
 'model.layers.10': 0,
 'model.layers.11': 0,
 'model.layers.12': 0,
 'model.layers.13': 1,
 'model.layers.14': 1,
 'model.layers.15': 1,
 'model.layers.16': 1,
 'model.layers.17': 1,
 'model.layers.18': 1,
 'model.layers.19': 1,
 'model.layers.20': 1,
 'model.layers.21': 1,
 'model.layers.22': 1,
 'model.layers.23': 1,
 'model.layers.24': 1,
 'model.layers.25': 1,
 'model.layers.26': 1,
 'model.layers.27': 1,
 'model.layers.28': 1,
 'model.layers.29': 1,
 'model.layers.30': 1,
 'model.layers.31': 1,
 'model.norm': 1,
 'lm_head': 1}

# Pipeline

In [17]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.bos_token_id
]


### hugging face pipeline
pipe = pipeline(
    task = "text-generation",
    
    model = model,
    
    tokenizer = tokenizer,
#     pad_token_id = tokenizer.eos_token_id,
    eos_token_id = terminators,
    
    do_sample = True,
#     max_length = CFG.max_len,
    max_new_tokens = CFG.max_new_tokens,
    
    
    temperature = CFG.temperature,
    top_p = CFG.top_p,
    repetition_penalty = CFG.repetition_penalty,
)

### langchain pipeline
llm = HuggingFacePipeline(pipeline = pipe)

# Prompt Template

[Prompt format for Phi-3 instruct](https://huggingface.co/microsoft/Phi-3-mini-128k-instruct)

```
<|system|>
You are a helpful AI assistant.<|end|>
<|user|>
I am going to Paris, what should I see?<|end|>
<|assistant|>
```

In [18]:
prompt_template = """
<|system|>

You are an expert assistant that answers questions about machine learning and Large Language Models (LLMs).

You are given some extracted parts from machine learning papers along with a question.

If you don't know the answer, just say "I don't know." Don't try to make up an answer.

It is very important that you ALWAYS answer the question in the same language the question is in. Remember to always do that.

Use only the following pieces of context to answer the question at the end.

<|end|>

<|user|>

Context: {context}

Question is below. Remember to answer in the same language:

Question: {question}

<|end|>

<|assistant|>

"""


PROMPT = PromptTemplate(
    template = prompt_template, 
    input_variables = ["context", "question"]
)

# Retrieval chain

- Retrieval chain using previously created vector database

In [19]:
retriever = vectordb.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k": CFG.k}
)

In [20]:
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff", # map_reduce, map_rerank, stuff, refine
    retriever = retriever, 
    chain_type_kwargs = {"prompt": PROMPT},
    return_source_documents = True,
    verbose = False
)

# Post-process outputs

In [21]:
def wrap_text_preserve_newlines(text, width=1500):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_llm_response(llm_response):
    ans = wrap_text_preserve_newlines(llm_response['result'])
    
    sources_used = ' \n'.join(
        [
            source.metadata['source'].split('/')[-1][:-4]
            + ' - page: '
            + str(source.metadata['page'])
            for source in llm_response['source_documents']
        ]
    )
    
    ans = ans + '\n\nSources: \n' + sources_used
    
    ### return only the text after the pattern
    pattern = "<|assistant|>"
    index = ans.find(pattern)
    if index != -1:
        ans = ans[index + len(pattern):]    
    
    return ans.strip()

def llm_ans(query):
    start = time.time()
    
    llm_response = qa_chain.invoke(query)
    ans = process_llm_response(llm_response)
    
    end = time.time()

    time_elapsed = int(round(end - start, 0))
    time_elapsed_str = f'\n\nTime elapsed: {time_elapsed} s'
    return ans + time_elapsed_str

# Ask Questions

In [22]:
query = "Tell me about the scaling laws for language models"
result = llm_ans(query)
clear_output()
print(result)

In investigating the scaling laws for language models, it was found that there are certain patterns or tendencies related to how the performance of these models changes as their sizes increase. For instance, studies have shown that while fine-tuning can enhance individual system performances significantly, overall trends suggest improvements in capabilities across various types of models regardless of whether they were finely tuned or not. This indicates that simply enlarging the size of these models tends to yield better outcomes in terms of state-of-the-art achievements in multiple language understanding tasks. Furthermore, despite facing issues such as increased computational demands and necessities for higher quality training data, these extensive models still appear pivotal for creating versatile and broadly applicable language technologies.

Sources: 
2203.15556 - page: 2 
2203.15556 - page: 16 
2303.18223 - page: 87 
2101.03961 - page: 36 
2203.15556 - page: 17 
2005.14165 - pag